In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as c
from astropy.wcs import WCS
import astropy.coordinates as coord
import astropy.units as u
from pyutils import *
import astropy.io.fits as fits
import healpy as hp
from astropy.table import Table

%load_ext autoreload
%autoreload 2

In [ ]:
ROOT_FOLDER = "../bin/"
FILE = ROOT_FOLDER + "BGS_BRIGHT_full.dat.fits"

In [ ]:
BGS_N = hp.read_map(ROOT_FOLDER + 'BGS_BRIGHT_mapprops_healpix_nested_nside256_N.fits')
BGS_S = hp.read_map(ROOT_FOLDER + 'BGS_BRIGHT_mapprops_healpix_nested_nside256_S.fits')

In [ ]:
hp.mollview(
    BGS_N,
    coord=["G", "E"],
    title="Histogram equalized Ecliptic",
    unit="Galaxies",
    norm="hist",
    min=0,
    max=1,
)
hp.graticule()

In [ ]:
hp.mollview(
    BGS_S,
    coord=["G", "E"],
    title="Histogram equalized Ecliptic",
    unit="Galaxies",
    norm="hist",
    min=0,
    max=1,
)
hp.graticule()

   We need:

   'app_mag' CALCULATE IT. Find info for the bands on how to convert. 𝑚=22.5−2.5log10(flux) probably?

   'dec' GOT IT

   'g_r' GOT IT (but k-corr?)

   'ra' GOT IT

   'z_obs' GOT IT

   If warnings say multi fits, consider it LOST. We can actually use that info later if available.

Healpix for geometric info, hopefully containing 3pass info.

   fiber info

In [ ]:
u_table = Table.read(FILE, format='fits')


In [ ]:
u_table.colnames

In [ ]:
u_table.keep_columns[
    "RA", 
    "DEC", 
    "Z", 
    "GALDEPTH_R", 
    "GALDEPTH_Z", 
    "FLUX_R", 
    "FLUX_Z", 
    "MW_TRANSMISSION_R",
    "MW_TRANSMISSION_Z",
    "TARGET_STATE",
    "SPECTYPE",
    "Z_not4clus"
]

In [ ]:
u_table.masked

In [ ]:
plt.hist(u_table['Z_not4clus'], bins=50)
plt.title("Z_not4clus")
plt.yscale('log')
print(np.min(u_table['Z_not4clus']), np.max(u_table['Z_not4clus']))
print(u_table['Z_not4clus'].mask)

In [ ]:
print(np.unique(u_table['ZWARN']))
print(np.unique(u_table['ZWARN_MTL']))
print(np.unique(u_table['SPECTYPE']))
print(np.unique(u_table['NTILE']))
print(np.unique(u_table['TARGET_STATE']))

In [ ]:
u_table.add_index(['TARGETID'])

## Get the galaxy data we actually need

In [ ]:
# TODO ensure this is right
def get_app_mag(FLUX_R):
    return 22.5 - 2.5*np.log10(FLUX_R)

In [ ]:
APP_MAG_CUT = 19.5
Z_MIN = 0.001
Z_MAX = 0.8
obj_type = u_table['SPECTYPE'].data.data
dec = u_table['DEC']
ra = u_table['RA']
z_obs = u_table['Z_not4clus'].data.data
target_id = u_table['TARGETID']
flux_r = u_table['FLUX_R']
app_mag = get_app_mag(u_table['FLUX_R'])
p_obs = u_table['PROB_OBS']
unobserved = u_table['ZWARN'] == 999999
deltachi2 = u_table['DELTACHI2'].data.data

before_count = len(dec)
print(before_count, "objects in FITS file")

# Make filter array (True/False values)
three_pass_filter = u_table['NTILE'] >= 3 # 3pass coverage
galaxy_filter = np.logical_or(obj_type == b'GALAXY', obj_type == b'')
app_mag_filter = app_mag < APP_MAG_CUT
redshift_filter = z_obs > Z_MIN
redshift_hi_filter = z_obs < Z_MAX
deltachi2_filter = deltachi2 > 40
keep = np.all([three_pass_filter, galaxy_filter, app_mag_filter, redshift_filter, redshift_hi_filter, deltachi2_filter], axis=0)
#keep = np.all([galaxy_filter], axis=0)

obj_type = obj_type[keep]
dec = dec[keep]
ra = ra[keep]
z_obs = z_obs[keep]
target_id = target_id[keep]
flux_r = flux_r[keep]
app_mag = app_mag[keep]
p_obs = p_obs[keep]
unobserved = unobserved[keep]
deltachi2 = deltachi2[keep]

after_count = len(dec)

print(f"Kept {after_count} of {before_count} rows.")
print(f'{unobserved.sum() } galaxies are unobserved')
print(f'{100*unobserved.sum() / len(unobserved) :.1f}% galaxies are unobserved')


In [ ]:
plt.hist(obj_type)

In [ ]:
plt.figure(figsize=(32,8))
plt.hist(u_table['TARGET_STATE'])
plt.title('Target State')
plt.yscale('log')

In [ ]:
print(np.min(z_obs))
print(np.max(z_obs))

In [ ]:
u_table['Z_not4clus'].mask[0:10]

In [ ]:
u_table['ZWARN'][0:10]

In [ ]:
plt.hist(u_table['ZWARN'])
plt.title('ZWARN')
plt.yscale('log')

In [ ]:
plt.hist(u_table['ZWARN_MTL'])
plt.title('ZWARN_MTL')
plt.yscale('log')

In [ ]:
plt.hist(z_obs, bins=50)
plt.title('Redshift')
plt.yscale('log')

In [ ]:
plt.hist(p_obs, bins=50)
plt.title('Probabilty to Observe')

In [ ]:
plt.hist(app_mag, bins=50)
plt.yscale('log')

In [ ]:
plt.hist(get_app_mag(u_table['FLUX_G']) - get_app_mag(u_table['FLUX_R']) , bins=20)
plt.title('g-r')
plt.yscale('log')

In [ ]:
print(estimate_frac_area(ra, dec))

In [ ]:
fig = make_map(ra, dec)